In [24]:
import random
import time
import numpy as np
import sys
from numpy import linspace

def corountine(func):
    def start(*args, **kwargs):
        g = func(*args, **kwargs)
        g.__next__()
        return g
    return start

def print_header():
    "Output results-header"
    f = open("out.txt", "w")
    f.write("%d\n" % N)
    #number of points in printing templates
    f.write("%d\n" % TMPN)
    
    for t in range(M):
        f.write("%d%s" % (glambda[t], ","))
        
    f.write("%d\n" % (glambda[M]))
    f.close()

def print_results(in_seq):
    "Output results"
    f = open("out.txt", "a")
    k = 0
    for i in range(N - 2):
        if in_seq[i].cid == template[k]:
            f.write("%f%s" % (in_seq[i].st, ","))
            k += 1
    f.write("%f\n" % (in_seq[N - 1].st))
    f.close()
    
@corountine
def server(i):
    ST = 0                  #sojourn time for the previous client
    item = None
    while True:
        item = (yield item) #get item
        if item == None:
            ST = 0
            continue
        waiting_time = max(0.0, ST - item.st - item.tau)
        item.st += random.expovariate(glambda[i + 1]) + waiting_time
        ST = item.st
        
def producer():
    results = []
    i = 0
    while True:
        if i == N: 
            break
        c = Client(i, 0., 0.)
        if i != 0:
            c.tau = random.expovariate(glambda[0])
        i += 1
        for s in p:
            c = s.send(c)
        results += [c]
    for s in p:
        c = s.send(None)  #final signal
    return results

class Client(object):
    def __init__(self, cid, st, tau):
        self.cid = cid
        self.st = st
        self.tau = tau
    def params(self):
        return (self.cid, self.st, self.tau)
    
stt = time.time()

N = 100         #Clients
M = 2          #Servers

#input/service frequency

glambda = [30000] + [i for i in linspace(2500, 5000, M)]
MKS = 3      #Monte Carlo simulation results

##Number of points in the printing template
TMPN = 10

##printing template
template = list(map(int, linspace(0, N - 1, TMPN)))

print_header()

p = []

for i in range(M):
    p += [server(i)]
    
for i in range(MKS):
    print_header()
    print_results(producer())
    print("Step = %d" % i)
    
sys.stdout.write("Processing time: %d\n" % int(time.time() - stt))
            

Step = 0
Step = 1
Step = 2
Processing time: 0
